In [1]:
#importing
import dbt_functions as dbt
import database_functions as database
import tab_functions as tab
import pandas as pd
from my_variables import *


In [2]:
#Authenticating against Snowflake and Tableau.

snow_auth = database.authenticate_snowflake(snow_u, snow_p, snow_account, snow_db)
tab_auth = tab.authenticate_tableau(tab_url,access_token,site_name,token_name)
oracle_auth = database.authenticate_oracle(ora_u=oracle_u, ora_p=oracle_p, ora_ip = oracle_db, ora_service= oracle_service)
dbt_account_id = dbt.get_account_id(dbt_token)
tab_token = tab_auth[0]
site_id = tab_auth[1]

In [3]:
#Run tab functions to get infos about databases used in Tableau
#oracle_infos = tab.get_table_infos(tab_url, oracle_db, tab_token)
snow_infos = tab.get_table_infos(tab_url, snow_db, tab_token)

In [23]:
#Get all current models running on dbt.
dbt_jobs = dbt.get_jobs(dbt_account_id,dbt_token)
dbt_models = dbt.get_models_information(dbt_token,'115014')

0    False
1    False
2    False
3    False
dtype: bool

In [49]:
#For each snowflake table on Tableau, add a data quality warning.
for index, row in snow_infos.iterrows():
    #Match dbt model with snowflake table
    if row['table_schema'] == dbt_models['schema'][0]:
        try:
            #Add a quality warning to Tableau.
            print(tab_url, site_id, row['table_luid'], data_quality_message.item() ,tab_token)
            model_part = dbt_models.loc[(dbt_models['schema'].str.contains(row['table_schema']))&(dbt_models['alias']==row['table_name'].lower())]
            data_quality_message = 'The data was last updated on ' + model_part['executeCompletedAt'] + ' and was ' + model_part['status'] + 'ful. The run took ' + str(model_part['runElapsedTime'].values) + ' seconds.'
            tab.add_data_quality_warning(tab_url, site_id, row['table_luid'], data_quality_message.item() ,tab_token)
            print('Added quality warning for ' + row['table_schema'] + '-' + row['table_name'])
        #Create message that is displayed in Tableau.
        except:
            print('no matching model found for ' + row['table_schema'] + '-' + row['table_name'])

no matching model found
no matching model found
no matching model found
no matching model found


In [7]:
#Script for getting a list of descriptions from Oracle and joining them with the matching columns in Tableau
#for index, row in oracle_infos.iterrows():
#    oracle_comment = database.get_oracle_descriptions(oracle_auth, row["table_name"])
#    table_columns = tab.get_list_of_columns(tab_url, row["table_luid"], site_id, tab_token)
#    joined = tab.add_comments_to_tab_table(table_columns, oracle_comment)
#    tab.update_table_descriptions(tab_url,site_id,joined,row["table_luid"],tab_token)

In [8]:
#Script for getting a list of descriptions from Snowflake and joining them with the matching columns in Tableau
for index, row in snow_infos.iterrows():
    snow_comment = database.get_snow_descriptions(snow_auth, row["table_name"], row["table_schema"])
    table_columns = tab.get_list_of_columns(tab_url, row["table_luid"], site_id, tab_token)
    joined = tab.add_comments_to_tab_table(table_columns, snow_comment)
    tab.update_table_descriptions(tab_url,site_id,joined,row["table_luid"],tab_token)

In [10]:
tab.logout(tab_url, tab_token)

Signed out successfully!
